# Klavis Poste Email MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Poste Email Toolathlon** (within local sandbox):
- **Acquire** a local sandbox with the `poste_email_toolathlon` MCP server
- **Get** sandbox details *(optional — acquire already returns everything)*
- **Call MCP tools** like `import_emails`, `get_emails`, and `send_email` using the **Python MCP SDK**
- **Release** the sandbox

> **Key differences from regular sandboxes:**
> - Email MCP requires access to a local environment (SMTP/IMAP), so it uses the **local sandbox** (`/local-sandbox`) API.
> - Each email MCP has a corresponding IP, SMTP port, and IMAP port — available in the `auth_data` field.
> - The user must specify their own email address, password, and name. Pass these as `x-email-config` (base64-encoded JSON) in the headers when calling MCP tools.

## 1. Setup

In [ ]:
import os
import json
import base64
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "poste_email_toolathlon"

## 2. Acquire a Local Sandbox with Poste Email

`POST /local-sandbox` — Acquire a local sandbox VM with the `poste_email_toolathlon` MCP server.

Unlike regular sandboxes (`/sandbox`), the local sandbox API spins up a VM hosting local MCP servers that need direct network access (SMTP/IMAP).

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/local-sandbox",
        headers=headers,
        json={"server_names": ["poste_email_toolathlon"], "benchmark": "Toolathlon"}
    )

acquire_data = resp.json()
LOCAL_SANDBOX_ID = acquire_data["local_sandbox_id"]

# Extract the email MCP server info directly from the acquire response
email_server = next(
    s for s in acquire_data["servers"]
    if s["server_name"] == "poste_email_toolathlon"
)

MCP_SERVER_URL = email_server["mcp_server_url"]
AUTH_DATA = email_server["auth_data"]

print(f"Local Sandbox ID: {LOCAL_SANDBOX_ID}")
print(f"MCP Server URL: {MCP_SERVER_URL}")
print(f"Email server config: {json.dumps(AUTH_DATA, indent=2)}")

## 3. Get Sandbox Details *(Optional)*

> The acquire response already returns all server details. These endpoints are useful if you need to re-fetch info later.

`GET /local-sandbox/{local_sandbox_id}` — Get details for a specific local sandbox

In [ ]:
# Get details for our sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/local-sandbox/{LOCAL_SANDBOX_ID}",
        headers=headers
    )
sandbox_info = resp.json()
print(json.dumps(sandbox_info, indent=2))

## 4. Call MCP Tools via the Python MCP SDK

The email MCP server runs inside the local sandbox and speaks the MCP Streamable HTTP protocol.

We pass the `x-email-config` header (base64-encoded JSON) so the MCP server knows which email account to operate on. The header value contains the user's **email**, **password**, and **name** — these are user-specified, not from `auth_data`.

`auth_data` from the acquire response only contains connection details (IP, SMTP port, IMAP port).

Since `streamable_http_client` does not accept `headers` directly, we create a pre-configured `httpx.AsyncClient` with the required headers and pass it via `http_client=`.

In [ ]:
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

# User-specified email credentials (not from auth_data)
# auth_data only contains connection info: IP, SMTP port, IMAP port
EMAIL_CONFIG = {
    "email": "micheller@mcp.com",
    "password": "michelle_60R",
    "name": "Ronald Kelly",
}

# Prepare the x-email-config header: base64-encode the email config JSON
email_config_b64 = base64.b64encode(json.dumps(EMAIL_CONFIG).encode()).decode()
mcp_headers = {"x-email-config": email_config_b64}

# Create a pre-configured httpx.AsyncClient with the MCP headers.
# streamable_http_client accepts http_client= (not headers=) for custom HTTP settings.
mcp_http_client = httpx.AsyncClient(headers=mcp_headers, timeout=httpx.Timeout(120))

print(f"MCP Server URL: {MCP_SERVER_URL}")
print(f"Email: {EMAIL_CONFIG['email']}")
print(f"x-email-config prepared (base64, {len(email_config_b64)} chars)")

### 4a. Check Connection

Verify that the IMAP and SMTP connections are working.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("check_connection", {})
        for content in result.content:
            print(content.text)

### 4b. Import Emails via MCP Tool

Use the `import_emails` MCP tool to upload emails directly into the email server. This accepts a JSON string with an `"emails"` key.

In [ ]:
# Import emails via the MCP tool
# The tool accepts a JSON string with an "emails" key containing a list of email objects
import_data = {
    "emails": [
        {
            "email_id": "1",
            "subject": "Quick sync needed",
            "from_addr": "alexander.hughes@mcp.com",
            "to_addr": EMAIL_CONFIG["email"],
            "cc_addr": None,
            "bcc_addr": None,
            "date": "44 days before current date",
            "message_id": "<cc6dfcca.12001@mcp.com>",
            "body_text": "\nHi! Can we sync on the project status? The deadline is 41 days before current date.Best,Team\n",
            "body_html": "<html><body><p>Hi! Can we sync on the project status? The deadline is <strong>41 days before current date</strong>.</p><p>Best,<br>Team</p></body></html>",
            "is_read": True,
            "is_important": False,
            "folder": "INBOX",
            "attachments": []
        },
        {
            "email_id": "2",
            "subject": "Weekly Update #150",
            "from_addr": "newsletter150@publication.com",
            "to_addr": EMAIL_CONFIG["email"],
            "cc_addr": None,
            "bcc_addr": None,
            "date": "5 days before current date",
            "message_id": "<620977df.12002@mcp.com>",
            "body_text": "\nThis Week's NewsPublished on 5 days before current dateTop stories this week...\n",
            "body_html": "<html><body><div style='font-family: Arial;'><h2>This Week's News</h2><p>Published on <strong>5 days before current date</strong></p><p>Top stories this week...</p></div></body></html>",
            "is_read": True,
            "is_important": False,
            "folder": "INBOX",
            "attachments": []
        }
    ]
}

async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("import_emails", {
            "json_string": json.dumps(import_data)
        })
        for content in result.content:
            print(content.text)

### 4c. List Folders

List all email folders on the server.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("list_folders", {})
        for content in result.content:
            print(content.text)

### 4d. Get Emails from INBOX

Retrieve emails from the INBOX folder.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("get_emails", {
            "folder": "INBOX",
            "limit": 10
        })
        for content in result.content:
            print(content.text)

### 4e. Send an Email

Send a new email using the SMTP server in the sandbox.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("send_email", {
            "to": EMAIL_CONFIG["email"],
            "subject": "Test email from Klavis Sandbox",
            "body": "Hello!\n\nThis is a test email sent via the Poste Email MCP sandbox.\n\nBest regards"
        })
        for content in result.content:
            print(content.text)

### 4f. Read the Sent Email

Read the email that was just sent.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL, http_client=mcp_http_client) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        # Fetch the latest emails from INBOX to find the one we just sent
        result = await session.call_tool("get_emails", {
            "folder": "Sent",
            "limit": 1
        })
        for content in result.content:
            print(content.text)

## 5. Release Local Sandbox

`DELETE /local-sandbox/{local_sandbox_id}` — Release the local sandbox VM and all associated servers

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/local-sandbox/{LOCAL_SANDBOX_ID}",
        headers=headers
    )

print(resp.json())